In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder


### 시점 구분

+ (LC 방문전)---(LC 방문)---(대출 실행)--->
1. LC 방문 전 발생하는 데이터
    + 대출 신청 전 fico 점수, 리볼빙, dti, 모기지론 여부, 근속년수 등
    + 예상 부도율 예측 모델이 작동(부도 상태 분류 모델로 부도율 예측)
        + 모델은 1종오류와 2종오류를 최소화 하는 것을 목적함수로 튜닝
2. LC에 방문해 생기는 데이터
    + 신용등급, 대출 승인 금액, 대출 이자율, 매월 상환 금액 등
    + 적정 이자율 예측 모델이 작동(예상 부도율과 Fully Paid 케이스의 대출 이자율을 타겟변수로 모델 학습)
        + Fully Paid 케이스에 책정된 대출 이자율이 부도가 나지 않을 만큼 올바르게 책정된 이자율이라는 가정
        + 모델은 샤프비율을 최대화 하는 것을 목적함수로 튜닝
3. 대출이 실행 시점에 생기는 데이터
    + 실제 대출 금액
4. 대출 실행 이후 생기는 데이터
    + 현재까지 상환한 금액, 채권 추심 금액 등

### 1. 방문전 발생하는 데이터 feature selection(44개 + One hot encoding)


| Features                 |                            |                         |                              |
|--------------------------|----------------------------|-------------------------|------------------------------|
| term                     | emp_length                 | annual_inc              | dti                          |
| delinq_2yrs              | fico_fico_avg              | inq_last_6mths          | open_acc                     |
| pub_rec                  | revol_bal                  | revol_util              | total_acc                    |
| acc_now_delinq           | tot_cur_bal                | mths_since_rcnt_il      | total_bal_il                 |
| il_util                  | max_bal_bc                 | all_util                | total_rev_hi_lim             |
| total_cu_tl              | avg_cur_bal                | bc_open_to_buy          | bc_util                      |
| chargeoff_within_12_mths | mort_acc                   | num_accts_ever_120_pd   | num_actv_rev_tl              |
| num_bc_sats              | num_bc_tl                  | num_op_rev_tl           | num_rev_accts                |
| num_rev_tl_bal_gt_0      | num_sats                   | pct_tl_nvr_dlq          | percent_bc_gt_75             |
| pub_rec_bankruptcies     | tax_liens                  | tot_hi_cred_lim         | total_bal_ex_mort            |
| total_bc_limit           | total_il_high_credit_limit | home_ownership(One-hot) | verification_status(One-hot) |

### 해당 feature들로 loan_status, loan_status_prop 예측. loan_status_prop는 데이터에는 없는 변수로, 모델의 laon_status 분류 확률값을 의미

In [13]:
df = pd.read_csv("./data/for_grade_data.csv", low_memory=True)
df.head()


FileNotFoundError: [Errno 2] No such file or directory: './data/for_grade_data.csv'

In [3]:
features = [
    'term', 'emp_length', 'annual_inc', 'dti', 'delinq_2yrs', 'fico_avg',
    'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc',
    'acc_now_delinq', 'tot_cur_bal', 'mths_since_rcnt_il', 'total_bal_il', 'il_util',
    'max_bal_bc', 'all_util', 'total_rev_hi_lim', 'total_cu_tl', 'avg_cur_bal',
    'bc_open_to_buy', 'bc_util', 'chargeoff_within_12_mths', 'mort_acc',
    'num_accts_ever_120_pd', 'num_actv_rev_tl', 'num_bc_sats', 'num_bc_tl',
    'num_op_rev_tl', 'num_rev_accts', 'num_rev_tl_bal_gt_0', 'num_sats', 'pct_tl_nvr_dlq',
    'percent_bc_gt_75', 'pub_rec_bankruptcies', 'tax_liens', 'tot_hi_cred_lim',
    'total_bal_ex_mort', 'total_bc_limit', 'total_il_high_credit_limit',
    'home_ownership_MORTGAGE', "home_ownership_NONE", "home_ownership_OTHER", "home_ownership_OWN", "home_ownership_RENT", "verification_status_Source Verified", "verification_status_Verified", "loan_status"
]


In [4]:
df = df[features]

df.info()
# 현재 칼럼 수 : 50개
print("현재 행 개수:", df.shape[0])
# 현재 행 수 : 1755294개


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1755294 entries, 0 to 1755293
Data columns (total 50 columns):
 #   Column                               Dtype  
---  ------                               -----  
 0   term                                 float64
 1   emp_length                           float64
 2   annual_inc                           float64
 3   dti                                  float64
 4   delinq_2yrs                          float64
 5   fico_avg                             float64
 6   inq_last_6mths                       float64
 7   open_acc                             float64
 8   pub_rec                              float64
 9   revol_bal                            float64
 10  revol_util                           float64
 11  total_acc                            float64
 12  acc_now_delinq                       float64
 13  tot_cur_bal                          float64
 14  mths_since_rcnt_il                   float64
 15  total_bal_il                    

In [5]:
# loan_status : 부도 여부, 타겟 변수
# non-default = 0, default = 1
# 'Fully Paid'와 'Charged Off'가 아닌 행 제거
df = df[df['loan_status'].isin(['Fully Paid', 'Charged Off'])]
df['loan_status'] = np.where(df['loan_status'] == 'Fully Paid', 0, 1)


In [6]:
df.info()
# 현재 칼럼 수 : 50개
print("현재 행 개수:", df.shape[0])
# 현재 행 수 : 1115888개


<class 'pandas.core.frame.DataFrame'>
Index: 1115888 entries, 0 to 1755292
Data columns (total 50 columns):
 #   Column                               Non-Null Count    Dtype  
---  ------                               --------------    -----  
 0   term                                 1115888 non-null  float64
 1   emp_length                           1115888 non-null  float64
 2   annual_inc                           1115888 non-null  float64
 3   dti                                  1115243 non-null  float64
 4   delinq_2yrs                          1115888 non-null  float64
 5   fico_avg                             1115888 non-null  float64
 6   inq_last_6mths                       1115888 non-null  float64
 7   open_acc                             1115888 non-null  float64
 8   pub_rec                              1115888 non-null  float64
 9   revol_bal                            1115888 non-null  float64
 10  revol_util                           1115888 non-null  float64
 11  tot

In [7]:
# 각 컬럼의 결측치 개수 계산
missing_counts = df.isnull().sum()

# 결측치가 있는 컬럼만 선택
missing_counts = missing_counts[missing_counts > 0]

# 결측치 비율 계산 (전체 행 수에 대한 비율)
missing_ratios = missing_counts / len(df)

# 결측치 개수와 비율을 하나의 DataFrame으로 생성한 후, 내림차순 정렬
missing_df = pd.DataFrame({
    'Missing Count': missing_counts,
    'Missing Ratio': missing_ratios
}).sort_values(by='Missing Count', ascending=False)

missing_df

,Missing Count,Missing Ratio
il_util,593266,0.531654
mths_since_rcnt_il,525278,0.470726
all_util,507913,0.455165
total_cu_tl,507831,0.455091
total_bal_il,507831,0.455091
max_bal_bc,507831,0.455091
dti,645,0.000578
pub_rec_bankruptcies,439,0.000393
chargeoff_within_12_mths,38,0.000034
tax_liens,29,0.000026


In [8]:
# 40% 이상 결측치를 가지는 칼럼 drop
df = df.loc[:, (df.isnull().mean() < 0.4) | (df.columns == "loan_status")]

df.info()
# 현재 칼럼 수 : 44개
print("현재 행 개수:", df.shape[0])
# 현재 행 수 : 1115888개

<class 'pandas.core.frame.DataFrame'>
Index: 1115888 entries, 0 to 1755292
Data columns (total 44 columns):
 #   Column                               Non-Null Count    Dtype  
---  ------                               --------------    -----  
 0   term                                 1115888 non-null  float64
 1   emp_length                           1115888 non-null  float64
 2   annual_inc                           1115888 non-null  float64
 3   dti                                  1115243 non-null  float64
 4   delinq_2yrs                          1115888 non-null  float64
 5   fico_avg                             1115888 non-null  float64
 6   inq_last_6mths                       1115888 non-null  float64
 7   open_acc                             1115888 non-null  float64
 8   pub_rec                              1115888 non-null  float64
 9   revol_bal                            1115888 non-null  float64
 10  revol_util                           1115888 non-null  float64
 11  tot

In [9]:
# 각 컬럼의 결측치 개수 계산
missing_counts = df.isnull().sum()

# 결측치가 있는 컬럼만 선택
missing_counts = missing_counts[missing_counts > 0]

# 결측치 비율 계산 (전체 행 수에 대한 비율)
missing_ratios = missing_counts / len(df)

# 결측치 개수와 비율을 하나의 DataFrame으로 생성한 후, 내림차순 정렬
missing_df = pd.DataFrame({
    'Missing Count': missing_counts,
    'Missing Ratio': missing_ratios
}).sort_values(by='Missing Count', ascending=False)

print(missing_df)

num_missing_rows = df.isnull().any(axis=1).sum()
print("결측치가 있는 행 개수:", num_missing_rows)
# 결측치 데이터 비율이 매우 적어 채우지 않고 그냥 드랍해버려도 될듯

                          Missing Count  Missing Ratio
dti                                 645       0.000578
pub_rec_bankruptcies                439       0.000393
chargeoff_within_12_mths             38       0.000034
tax_liens                            29       0.000026
결측치가 있는 행 개수: 1085


In [10]:
# 결측치가 있는 행 모두 제거 (원본 df에 바로 적용)
df.dropna(inplace=True)


In [11]:
df.info()
# 현재 칼럼 수 : 44개
print("현재 행 개수:", df.shape[0])
# 현재 행 수 : 1114803개

df.to_csv("./data/fst_timeline.csv", index=False)

<class 'pandas.core.frame.DataFrame'>
Index: 1114803 entries, 0 to 1755292
Data columns (total 44 columns):
 #   Column                               Non-Null Count    Dtype  
---  ------                               --------------    -----  
 0   term                                 1114803 non-null  float64
 1   emp_length                           1114803 non-null  float64
 2   annual_inc                           1114803 non-null  float64
 3   dti                                  1114803 non-null  float64
 4   delinq_2yrs                          1114803 non-null  float64
 5   fico_avg                             1114803 non-null  float64
 6   inq_last_6mths                       1114803 non-null  float64
 7   open_acc                             1114803 non-null  float64
 8   pub_rec                              1114803 non-null  float64
 9   revol_bal                            1114803 non-null  float64
 10  revol_util                           1114803 non-null  float64
 11  tot

### 시점 구분

+ (LC 방문전)---(LC 방문)---(대출 실행)--->
1. LC 방문 전 발생하는 데이터
    + 대출 신청 전 fico 점수, 리볼빙, dti, 모기지론 여부, 근속년수 등
    + 예상 부도율 예측 모델이 작동(부도 상태 분류 모델로 부도율 예측)
        + 모델은 1종오류와 2종오류를 최소화 하는 것을 목적함수로 튜닝
2. LC에 방문해 생기는 데이터
    + 신용등급, 대출 승인 금액, 대출 이자율, 대출자가 신청한 금액 등
    + 적정 이자율 예측 모델이 작동(예상 부도율과 Fully Paid 케이스의 대출 이자율을 타겟변수로 모델 학습)
        + Fully Paid 케이스에 책정된 대출 이자율이 부도가 나지 않을 만큼 올바르게 책정된 이자율이라는 가정
        + 모델은 샤프비율을 최대화 하는 것을 목적함수로 튜닝
3. 대출이 실행 시점에 생기는 데이터
    + 실제 대출 금액
4. 대출 실행 이후 생기는 데이터
    + 현재까지 상환한 금액, 채권 추심 금액 등

### 2. LC방문시 발생하는 데이터 feature selection(44개 + One hot encoding), 이자율 예측 데이터


| Feature                  |                            |                         |                              |
|--------------------------|----------------------------|-------------------------|------------------------------|
| term                     | emp_length                 | annual_inc              | dti                          |
| delinq_2yrs              | fico_fico_avg              | inq_last_6mths          | open_acc                     |
| pub_rec                  | revol_bal                  | revol_util              | total_acc                    |
| acc_now_delinq           | tot_cur_bal                | mths_since_rcnt_il      | total_bal_il                 |
| il_util                  | max_bal_bc                 | all_util                | total_rev_hi_lim             |
| total_cu_tl              | avg_cur_bal                | bc_open_to_buy          | bc_util                      |
| chargeoff_within_12_mths | mort_acc                   | num_accts_ever_120_pd   | num_actv_rev_tl              |
| num_bc_sats              | num_bc_tl                  | num_op_rev_tl           | num_rev_accts                |
| num_rev_tl_bal_gt_0      | num_sats                   | pct_tl_nvr_dlq          | percent_bc_gt_75             |
| pub_rec_bankruptcies     | tax_liens                  | tot_hi_cred_lim         | total_bal_ex_mort            |
| total_bc_limit           | total_il_high_credit_limit | home_ownership(One-hot) | verification_status(One-hot) |

### +

| Feature                  |                            |                         |                              |
|--------------------------|----------------------------|-------------------------|------------------------------|
| loan_status              | loan_status_prop           | grade                   | sub_grade                    |
| loan_amnt                |                            |                         |                              |

### 해당 feature들로 int_rate를 예측

In [12]:
df = pd.read_csv("./data/test_trd_timeline_scaled.csv", low_memory=False)
df.head()


,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,annual_inc,...,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,home_ownership_MORTGAGE,home_ownership_NONE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT,verification_status_Source Verified,verification_status_Verified
0,0.812055,0.812055,0.827113,0,0.031225,0.817396,0.000000,0.000000,0.636364,0.656528,...,0.700911,0.640823,0.740276,True,False,False,False,False,True,False
1,0.673551,0.673551,0.699705,1,0.389017,0.682735,0.333333,0.352941,0.090909,0.600222,...,0.636249,0.489734,0.592855,False,False,False,False,True,True,False
2,0.734053,0.734053,0.755360,0,0.000435,0.764872,0.000000,0.000000,0.181818,0.617340,...,0.656255,0.533546,0.678609,True,False,False,False,False,False,False
3,0.812055,0.812055,0.827113,0,0.325012,0.835438,0.166667,0.264706,0.181818,0.629166,...,0.728281,0.574761,0.754260,False,False,False,True,False,False,True
4,0.653631,0.653631,0.681382,0,0.362960,0.734915,0.333333,0.382353,0.363636,0.604569,...,0.803623,0.580972,0.817131,False,False,False,False,True,True,False


In [14]:
features = [
    'term', 'emp_length', 'annual_inc', 'dti', 'delinq_2yrs', 'fico_avg',
    'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc',
    'acc_now_delinq', 'tot_cur_bal',
    'total_rev_hi_lim', 'avg_cur_bal',
    'bc_open_to_buy', 'bc_util', 'chargeoff_within_12_mths', 'mort_acc',
    'num_accts_ever_120_pd', 'num_actv_rev_tl', 'num_bc_sats', 'num_bc_tl',
    'num_op_rev_tl', 'num_rev_accts', 'num_rev_tl_bal_gt_0', 'num_sats', 'pct_tl_nvr_dlq',
    'percent_bc_gt_75', 'pub_rec_bankruptcies', 'tax_liens', 'tot_hi_cred_lim',
    'total_bal_ex_mort', 'total_bc_limit', 'total_il_high_credit_limit',
    'home_ownership_MORTGAGE', "home_ownership_NONE", "home_ownership_OTHER", "home_ownership_OWN", "home_ownership_RENT", "verification_status_Source Verified", "verification_status_Verified", "loan_status", "grade", "sub_grade", "loan_amnt", "int_rate"
]


In [31]:
df = df[features]

df.info()
# 현재 칼럼 수 : 54개
print("현재 행 개수:", df.shape[0])
# 현재 행 수 : 1755294개


KeyError: "['mths_since_rcnt_il', 'total_bal_il', 'il_util', 'max_bal_bc', 'all_util', 'total_cu_tl'] not in index"

In [16]:
# loan_status : 부도 여부, 타겟 변수
# non-default = 0, default = 1
# 'Fully Paid'와 'Charged Off'가 아닌 행 제거
df = df[df['loan_status'].isin(['Fully Paid', 'Charged Off'])]
df['loan_status'] = np.where(df['loan_status'] == 'Fully Paid', 0, 1)


In [17]:
df.info()
# 현재 칼럼 수 : 54개
print("현재 행 개수:", df.shape[0])
# 현재 행 수 : 1115888개


<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 48 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   term                                 0 non-null      int64  
 1   emp_length                           0 non-null      float64
 2   annual_inc                           0 non-null      float64
 3   dti                                  0 non-null      float64
 4   delinq_2yrs                          0 non-null      int64  
 5   fico_avg                             0 non-null      float64
 6   inq_last_6mths                       0 non-null      int64  
 7   open_acc                             0 non-null      float64
 8   pub_rec                              0 non-null      int64  
 9   revol_bal                            0 non-null      float64
 10  revol_util                           0 non-null      float64
 11  total_acc                            0 non-null  

In [18]:
# 각 컬럼의 결측치 개수 계산
missing_counts = df.isnull().sum()

# 결측치가 있는 컬럼만 선택
missing_counts = missing_counts[missing_counts > 0]

# 결측치 비율 계산 (전체 행 수에 대한 비율)
missing_ratios = missing_counts / len(df)

# 결측치 개수와 비율을 하나의 DataFrame으로 생성한 후, 내림차순 정렬
missing_df = pd.DataFrame({
    'Missing Count': missing_counts,
    'Missing Ratio': missing_ratios
}).sort_values(by='Missing Count', ascending=False)

missing_df

,Missing Count,Missing Ratio


In [19]:
# 40% 이상 결측치를 가지는 칼럼 drop
df = df.loc[:, (df.isnull().mean() < 0.4) | (df.columns == "loan_status")]

df.info()
# 현재 칼럼 수 : 48개
print("현재 행 개수:", df.shape[0])
# 현재 행 수 : 1115888개

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 1 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   loan_status  0 non-null      int64
dtypes: int64(1)
memory usage: 0.0 bytes
현재 행 개수: 0


In [20]:
# 각 컬럼의 결측치 개수 계산
missing_counts = df.isnull().sum()

# 결측치가 있는 컬럼만 선택
missing_counts = missing_counts[missing_counts > 0]

# 결측치 비율 계산 (전체 행 수에 대한 비율)
missing_ratios = missing_counts / len(df)

# 결측치 개수와 비율을 하나의 DataFrame으로 생성한 후, 내림차순 정렬
missing_df = pd.DataFrame({
    'Missing Count': missing_counts,
    'Missing Ratio': missing_ratios
}).sort_values(by='Missing Count', ascending=False)

print(missing_df)
# 결측치 데이터 비율이 매우 적어 채우지 않고 그냥 드랍해버려도 될듯

num_missing_rows = df.isnull().any(axis=1).sum()
print("결측치가 있는 행 개수:", num_missing_rows)


Empty DataFrame
Columns: [Missing Count, Missing Ratio]
Index: []
결측치가 있는 행 개수: 0


In [20]:
# 결측치가 있는 행 모두 제거 (원본 df에 바로 적용)
df.dropna(inplace=True)


In [11]:
df.info()
# 현재 칼럼 수 : 48개
print("현재 행 개수:", df.shape[0])
# 현재 행 수 : 1114803개

df.to_csv("./data/sec_timeline.csv", index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 680570 entries, 0 to 680569
Data columns (total 48 columns):
 #   Column                               Non-Null Count   Dtype  
---  ------                               --------------   -----  
 0   term                                 680570 non-null  int64  
 1   emp_length                           680570 non-null  float64
 2   annual_inc                           680570 non-null  float64
 3   dti                                  680570 non-null  float64
 4   delinq_2yrs                          680570 non-null  int64  
 5   fico_avg                             680570 non-null  float64
 6   inq_last_6mths                       680570 non-null  int64  
 7   open_acc                             680570 non-null  float64
 8   pub_rec                              680570 non-null  int64  
 9   revol_bal                            680570 non-null  float64
 10  revol_util                           680570 non-null  float64
 11  total_acc    

### 2. LC방문시 발생하는 데이터 feature selection(44개 + One hot encoding), 회수율 예측 데이터 


| Feature                  |                            |                         |                              |
|--------------------------|----------------------------|-------------------------|------------------------------|
| term                     | emp_length                 | annual_inc              | dti                          |
| delinq_2yrs              | fico_fico_avg              | inq_last_6mths          | open_acc                     |
| pub_rec                  | revol_bal                  | revol_util              | total_acc                    |
| acc_now_delinq           | tot_cur_bal                | mths_since_rcnt_il      | total_bal_il                 |
| il_util                  | max_bal_bc                 | all_util                | total_rev_hi_lim             |
| total_cu_tl              | avg_cur_bal                | bc_open_to_buy          | bc_util                      |
| chargeoff_within_12_mths | mort_acc                   | num_accts_ever_120_pd   | num_actv_rev_tl              |
| num_bc_sats              | num_bc_tl                  | num_op_rev_tl           | num_rev_accts                |
| num_rev_tl_bal_gt_0      | num_sats                   | pct_tl_nvr_dlq          | percent_bc_gt_75             |
| pub_rec_bankruptcies     | tax_liens                  | tot_hi_cred_lim         | total_bal_ex_mort            |
| total_bc_limit           | total_il_high_credit_limit | home_ownership(One-hot) | verification_status(One-hot) |

### +

| Feature                  |                            |                         |                              |
|--------------------------|----------------------------|-------------------------|------------------------------|
| loan_status              | loan_status_prop           | grade                   | sub_grade                    |
| loan_amnt                |                            |                         |                              |

### 해당 feature들로 회수율을 예측
+ 회수율 = total_pymnt / (term * installment)


In [21]:
df = pd.read_csv("./data/lending_club_2020_test.csv", low_memory=False)
df.head()


,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,...,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag
0,3697367,20000.0,20000.0,20000.0,36 months,6.03%,608.72,A,A1,RANDALL J STOLL CPA PC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
1,154429912,14950.0,14950.0,14950.0,60 months,17.74%,377.53,C,C5,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
2,163922381,19125.0,19125.0,19125.0,36 months,8.81%,606.49,A,A5,Machine operator,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
3,13046178,12000.0,12000.0,12000.0,60 months,14.64%,283.22,C,C3,Tech Locator,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
4,64038304,16000.0,16000.0,16000.0,60 months,9.99%,339.88,B,B3,Managing Dentist,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N


In [22]:
# loan_status : 부도 여부, 타겟 변수
# non-default = 0, default = 1
# 'Fully Paid'와 'Charged Off'가 아닌 행 제거
df = df[df['loan_status'].isin(['Fully Paid', 'Charged Off'])]
df['loan_status'] = np.where(df['loan_status'] == 'Fully Paid', 0, 1)


In [23]:
features = [
    'term', 'emp_length', 'annual_inc', 'dti', 'delinq_2yrs', 'fico_avg',
    'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc',
    'acc_now_delinq', 'tot_cur_bal', 'mths_since_rcnt_il', 'total_bal_il', 'il_util',
    'max_bal_bc', 'all_util', 'total_rev_hi_lim', 'total_cu_tl', 'avg_cur_bal',
    'bc_open_to_buy', 'bc_util', 'chargeoff_within_12_mths', 'mort_acc',
    'num_accts_ever_120_pd', 'num_actv_rev_tl', 'num_bc_sats', 'num_bc_tl',
    'num_op_rev_tl', 'num_rev_accts', 'num_rev_tl_bal_gt_0', 'num_sats', 'pct_tl_nvr_dlq',
    'percent_bc_gt_75', 'pub_rec_bankruptcies', 'tax_liens', 'tot_hi_cred_lim',
    'total_bal_ex_mort', 'total_bc_limit', 'total_il_high_credit_limit',
    'home_ownership_MORTGAGE', "home_ownership_NONE", "home_ownership_OTHER", "home_ownership_OWN", "home_ownership_RENT", "verification_status_Source Verified", "verification_status_Verified", "loan_status", "grade", "sub_grade", "loan_amnt", "total_pymnt", "installment", "int_rate"
]


In [24]:
# Qualitative var labeling(nominal var. without ordinal var)
df = pd.get_dummies(df, columns=["home_ownership", "verification_status"], drop_first=True)

# fico_range_low
# data cleansing. add avg fico score column 
insert_loc = df.columns.get_loc('fico_range_low')
df.insert(insert_loc, 'fico_avg', (df['fico_range_low'] + df['fico_range_high']) / 2)
df.drop(columns=['fico_range_low', 'fico_range_high'])

df = df[features]

In [25]:
# term
# categorical variable Labeling
label_mapping = {
    ' 36 months': 36,
    ' 60 months': 60
}

# emp_length
# 2 way of emp_length mapping.
# emp_length이 NaN인 값(무직)을 -1로 매핑, 나머지를 오름차순으로 0~10
# emp_length이 NaN인 값(무직)을 -1로 매핑, 나머지를 오름차순으로 1~11. 무직과 고용상태를 구분하기 위함
label_mapping = {
    '< 1 year': 0,
    '1 year': 1,
    '2 years': 2,
    '3 years': 3,
    '4 years': 4,
    '5 years': 5,
    '6 years': 6,
    '7 years': 7,
    '8 years': 8,
    '9 years': 9,
    '10+ years': 10
}

df['emp_length'] = df['emp_length'].map(label_mapping)
df['emp_length'] = df['emp_length'].fillna(-1)

# revol_util
# follow paper (Machine learning and artificial neural networks to construct P2P lending credit-scoring model: A case using Lending Club data), form change percentage form to decimal form
df["revol_util"] = df["revol_util"].str.rstrip("%").astype(float) / 100

# grade, sub_grade labeling
grade = np.array(df["grade"])
sub_grade = np.array(df["sub_grade"])

encoder = LabelEncoder()
grade_encoded = encoder.fit_transform(grade)
sub_grade_encoded = encoder.fit_transform(sub_grade)

df["grade"] = grade_encoded
df["sub_grade"] = sub_grade_encoded


In [26]:
df.info()
# 현재 칼럼 수 : 56개
print("현재 행 개수:", df.shape[0])
# 현재 행 수 : 1115888개


<class 'pandas.core.frame.DataFrame'>
Index: 744443 entries, 0 to 1170196
Data columns (total 56 columns):
 #   Column                               Non-Null Count   Dtype  
---  ------                               --------------   -----  
 0   term                                 744443 non-null  object 
 1   emp_length                           744443 non-null  float64
 2   annual_inc                           744443 non-null  float64
 3   dti                                  743980 non-null  float64
 4   delinq_2yrs                          744443 non-null  float64
 5   fico_avg                             744443 non-null  float64
 6   inq_last_6mths                       744442 non-null  float64
 7   open_acc                             744443 non-null  float64
 8   pub_rec                              744443 non-null  float64
 9   revol_bal                            744443 non-null  float64
 10  revol_util                           743885 non-null  float64
 11  total_acc        

In [32]:
# 각 컬럼의 결측치 개수 계산
missing_counts = df.isnull().sum()

# 결측치가 있는 컬럼만 선택
missing_counts = missing_counts[missing_counts > 0]

# 결측치 비율 계산 (전체 행 수에 대한 비율)
missing_ratios = missing_counts / len(df)

# 결측치 개수와 비율을 하나의 DataFrame으로 생성한 후, 내림차순 정렬
missing_df = pd.DataFrame({
    'Missing Count': missing_counts,
    'Missing Ratio': missing_ratios
}).sort_values(by='Missing Count', ascending=False)

missing_df

,Missing Count,Missing Ratio
dti,463,0.000622
pub_rec_bankruptcies,258,0.000347
chargeoff_within_12_mths,18,0.000024
tax_liens,10,0.000013
inq_last_6mths,1,0.000001


In [33]:
# 40% 이상 결측치를 가지는 칼럼 drop
df = df.loc[:, (df.isnull().mean() < 0.4) | (df.columns == "loan_status")]

df.info()
# 현재 칼럼 수 : 49개
print("현재 행 개수:", df.shape[0])
# 현재 행 수 : 1115888개

<class 'pandas.core.frame.DataFrame'>
Index: 744443 entries, 0 to 1170196
Data columns (total 50 columns):
 #   Column                               Non-Null Count   Dtype  
---  ------                               --------------   -----  
 0   term                                 744443 non-null  int64  
 1   emp_length                           744443 non-null  float64
 2   annual_inc                           744443 non-null  float64
 3   dti                                  743980 non-null  float64
 4   delinq_2yrs                          744443 non-null  float64
 5   fico_avg                             744443 non-null  float64
 6   inq_last_6mths                       744442 non-null  float64
 7   open_acc                             744443 non-null  float64
 8   pub_rec                              744443 non-null  float64
 9   revol_bal                            744443 non-null  float64
 10  revol_util                           744443 non-null  float64
 11  total_acc        

In [34]:
# 각 컬럼의 결측치 개수 계산
missing_counts = df.isnull().sum()

# 결측치가 있는 컬럼만 선택
missing_counts = missing_counts[missing_counts > 0]

# 결측치 비율 계산 (전체 행 수에 대한 비율)
missing_ratios = missing_counts / len(df)

# 결측치 개수와 비율을 하나의 DataFrame으로 생성한 후, 내림차순 정렬
missing_df = pd.DataFrame({
    'Missing Count': missing_counts,
    'Missing Ratio': missing_ratios
}).sort_values(by='Missing Count', ascending=False)

missing_df

,Missing Count,Missing Ratio
dti,463,0.000622
pub_rec_bankruptcies,258,0.000347
chargeoff_within_12_mths,18,0.000024
tax_liens,10,0.000013
inq_last_6mths,1,0.000001


In [30]:
# term
# categorical variable Labeling
label_mapping = {
    ' 36 months': 36,
    ' 60 months': 60
}
df['term'] = df['term'].map(label_mapping)

# bc_util
# 은행카드 한도 대비 잔액 비율
# 결측값은 평균값으로
df['bc_util'] = df['bc_util'] / 100.0
df['bc_util'] = df['bc_util'].fillna(df['bc_util'].mean())

# pct_tl_nvr_dlq : 연체 경험 없는 계좌 비율
# percent_bc_gt_75 : 한도 75% 초과 은행카드 계좌 비율
# 결측값은 평균으로
cols = ['pct_tl_nvr_dlq', 'percent_bc_gt_75']
df[cols] = df[cols].fillna(df[cols].mean())

# bc_open_to_buy
# 리볼빙 은행카드 사용 가능 한도
# 결측값은 중앙값으로
df["bc_open_to_buy"] = df["bc_open_to_buy"].fillna(df["bc_open_to_buy"].median())

# avg_cur_bal
# 모든 계좌의 평균 현재 잔액
# 결측값은 중앙값으로
df["avg_cur_bal"] = df["avg_cur_bal"].fillna(df["avg_cur_bal"].median())

# num_actv_bc_tl
# num_actv_rev_tl
# num_bc_sats
# num_bc_tl
# num_il_tl
# num_op_rev_tl
# num_rev_accts
# num_rev_tl_bal_gt_0
# num_sats
# 다 현재 계좌 수에 관련된 칼럼. 나중에 보고 좀 합쳐야할듯
# 결측값은 중앙값으로
cols = [
    'num_actv_rev_tl',
    'num_bc_sats',
    'num_bc_tl',
    'num_op_rev_tl',
    'num_rev_accts',
    'num_rev_tl_bal_gt_0',
    'num_sats'
]
df[cols] = df[cols].fillna(df[cols].median())

# tot_cur_bal
# 모든 계좌 현재 잔액 합계
# 결측값은 중앙값으로
df["tot_cur_bal"] = df["tot_cur_bal"].fillna(df["tot_cur_bal"].median())

# total_rev_hi_lim
# 리볼빙 총 한도
# 결측값은 중앙값으로
df["total_rev_hi_lim"] = df["total_rev_hi_lim"].fillna(df["total_rev_hi_lim"].median())

# num_tl_90g_dpd_24m : 최근 24개월 90일 이상 연체 계좌 수
# num_tl_op_past_12m : 최근 12개월 개설 계좌 수
# tot_hi_cred_lim : 총 최고 신용 한도
# total_bal_ex_mort : 주택담보 제외 총 잔액
# total_bc_limit : 은행카드 총 한도
# total_il_high_credit_limit : 할부 계좌 총 한도
# 결측값은 중앙값으로
cols = [
    'tot_hi_cred_lim',
    'total_bal_ex_mort',
    'total_bc_limit',
    'total_il_high_credit_limit'
]
df[cols] = df[cols].fillna(df[cols].median())

# num_accts_ever_120_pd
# 120일 이상 연체 경험 계좌 수
# 결측값은 중앙값으로
df["num_accts_ever_120_pd"] = df["num_accts_ever_120_pd"].fillna(df["num_accts_ever_120_pd"].median())

# mort_acc
# 주택담보대출 계좌 수
# 결측값은 중앙값으로
df["mort_acc"] = df["mort_acc"].fillna(df["mort_acc"].median())

# revol_util
# follow paper (Machine learning and artificial neural networks to construct P2P lending credit-scoring model: A case using Lending Club data), form change percentage form to decimal form
# 결측치는 평균값으로 채워넣는다
df['revol_util'] = df['revol_util'].fillna(df['revol_util'].mean())

In [35]:
# 각 컬럼의 결측치 개수 계산
missing_counts = df.isnull().sum()

# 결측치가 있는 컬럼만 선택
missing_counts = missing_counts[missing_counts > 0]

# 결측치 비율 계산 (전체 행 수에 대한 비율)
missing_ratios = missing_counts / len(df)

# 결측치 개수와 비율을 하나의 DataFrame으로 생성한 후, 내림차순 정렬
missing_df = pd.DataFrame({
    'Missing Count': missing_counts,
    'Missing Ratio': missing_ratios
}).sort_values(by='Missing Count', ascending=False)

print(missing_df)
# 결측치 데이터 비율이 매우 적어 채우지 않고 그냥 드랍해버려도 될듯

num_missing_rows = df.isnull().any(axis=1).sum()
print("결측치가 있는 행 개수:", num_missing_rows)


                          Missing Count  Missing Ratio
dti                                 463       0.000622
pub_rec_bankruptcies                258       0.000347
chargeoff_within_12_mths             18       0.000024
tax_liens                            10       0.000013
inq_last_6mths                        1       0.000001
결측치가 있는 행 개수: 723


In [37]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from tqdm import tqdm

# 예시: df는 원본 데이터프레임
# 데이터를 청크로 나누기 (여기서는 10개 청크로 나눕니다)
n_chunks = 1
chunk_size = int(np.ceil(len(df) / n_chunks))
imputer = KNNImputer(n_neighbors=3)
imputed_chunks = []

for i in tqdm(range(n_chunks), desc="Imputing chunks"):
    chunk = df.iloc[i*chunk_size : (i+1)*chunk_size]
    imputed_chunk = pd.DataFrame(imputer.fit_transform(chunk),
                                 columns=chunk.columns,
                                 index=chunk.index)
    imputed_chunks.append(imputed_chunk)

# 청크들을 다시 합칩니다.
df_imputed = pd.concat(imputed_chunks).sort_index()

print("\nKNN Imputation 후 데이터 일부:")
print(df_imputed.head())


Imputing chunks:   0%|          | 0/1 [00:00<?, ?it/s]


ValueError: could not convert string to float: '  6.03%'

In [38]:
df.info()
# 현재 칼럼 수 : 50개
print("현재 행 개수:", df.shape[0])
# 현재 행 수 : 1114803개

<class 'pandas.core.frame.DataFrame'>
Index: 1114803 entries, 0 to 1755293
Data columns (total 50 columns):
 #   Column                               Non-Null Count    Dtype  
---  ------                               --------------    -----  
 0   term                                 1114803 non-null  int64  
 1   emp_length                           1114803 non-null  float64
 2   annual_inc                           1114803 non-null  float64
 3   dti                                  1114803 non-null  float64
 4   delinq_2yrs                          1114803 non-null  float64
 5   fico_avg                             1114803 non-null  float64
 6   inq_last_6mths                       1114803 non-null  float64
 7   open_acc                             1114803 non-null  float64
 8   pub_rec                              1114803 non-null  float64
 9   revol_bal                            1114803 non-null  float64
 10  revol_util                           1114803 non-null  float64
 11  tot

In [39]:
df.head()
df.to_csv("./data/sec_timeline_recovery.csv", index=False)